<a href="https://colab.research.google.com/github/mvajhi/unsupervised-learning/blob/master/Deep%20learning/Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

files.upload();

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle

!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d aima138/17flowerclasses

Dataset URL: https://www.kaggle.com/datasets/aima138/17flowerclasses
License(s): unknown
  0% 0.00/57.8M [00:00<?, ?B/s]
100% 57.8M/57.8M [00:00<00:00, 1.33GB/s]


In [ ]:
!unzip 17flowerclasses.zip

Archive:  17flowerclasses.zip
  inflating: 17flowerclasses/test/Bluebell/image_0246.jpg  
  inflating: 17flowerclasses/test/Bluebell/image_0251.jpg  
  inflating: 17flowerclasses/test/Bluebell/image_0257.jpg  
  inflating: 17flowerclasses/test/Bluebell/image_0262.jpg  
  inflating: 17flowerclasses/test/Bluebell/image_0282.jpg  
  inflating: 17flowerclasses/test/Bluebell/image_0287.jpg  
  inflating: 17flowerclasses/test/Bluebell/image_0291.jpg  
  inflating: 17flowerclasses/test/Bluebell/image_0298.jpg  
  inflating: 17flowerclasses/test/Bluebell/image_0310.jpg  
  inflating: 17flowerclasses/test/Bluebell/image_0316.jpg  
  inflating: 17flowerclasses/test/ButterCup/image_1126.jpg  
  inflating: 17flowerclasses/test/ButterCup/image_1139.jpg  
  inflating: 17flowerclasses/test/ButterCup/image_1154.jpg  
  inflating: 17flowerclasses/test/ButterCup/image_1162.jpg  
  inflating: 17flowerclasses/test/ButterCup/image_1167.jpg  
  inflating: 17flowerclasses/test/ButterCup/image_1173.jpg  
  in

In [ ]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory('17flowerclasses/train')
test_ds = tf.keras.utils.image_dataset_from_directory('17flowerclasses/test')

print(train_ds.class_names)

Found 1190 files belonging to 17 classes.
Found 170 files belonging to 17 classes.
['Bluebell', 'ButterCup', 'ColtsFoot', 'Cowslip', 'Crocus', 'Daffodil', 'Daisy', 'Dandelion', 'Fritillary', 'Iris', 'LilyValley', 'Pansy', 'Snowdrop', 'Sunflower', 'Tigerlily', 'WindFlower', 'tulip']


In [ ]:
num_classes = len(train_ds.class_names)

data_augmentation = tf.keras.Sequential(
  [
    # tf.keras.layers.RandomFlip("horizontal", input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.2),
  ],
  name="data_augmentation"
)

heavy_data_augmentation = tf.keras.Sequential(
  [
    # این لایه‌ها را به ترتیب اعمال می‌کنیم
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.4), # چرخش با زاویه بیشتر
    tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2), # جابجایی بیشتر
    tf.keras.layers.RandomZoom(height_factor=0.3, width_factor=0.3), # زوم بیشتر

    # تغییرات رنگ و نور
    tf.keras.layers.RandomBrightness(factor=0.3),
    # tf.keras.layers.RandomContrast(factor=0.3),

    # این لایه به صورت تصادفی بخشی از تصویر را سیاه می‌کند (بسیار موثر)
    # layers.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3)) # در نسخه‌های جدیدتر TF
  ],
  name="heavy_data_augmentation"
)

data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(0.8),
        tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2), # جابجایی بیشتر
        tf.keras.layers.RandomZoom(0.2),
        tf.keras.layers.RandomBrightness(factor=0.3),
        tf.keras.layers.RandomContrast(0.2),
    ], name="data_augmentation")

model = tf.keras.Sequential([
    data_augmentation,
    tf.keras.layers.Rescaling(1./255, input_shape=(256, 256, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 5, activation='relu'),
    tf.keras.layers.Conv2D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 7, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=30
)

Epoch 1/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 207ms/step - accuracy: 0.0653 - loss: 3.1790 - val_accuracy: 0.1412 - val_loss: 2.4704
Epoch 2/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 193ms/step - accuracy: 0.1636 - loss: 2.4897 - val_accuracy: 0.2118 - val_loss: 2.3853
Epoch 3/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 192ms/step - accuracy: 0.2481 - loss: 2.2497 - val_accuracy: 0.1765 - val_loss: 2.4267
Epoch 4/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 199ms/step - accuracy: 0.2309 - loss: 2.2923 - val_accuracy: 0.3235 - val_loss: 2.1264
Epoch 5/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 192ms/step - accuracy: 0.2929 - loss: 2.0308 - val_accuracy: 0.2706 - val_loss: 2.0385
Epoch 6/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 0.2712 - loss: 2.0216 - val_accuracy: 0.2882 - val_loss: 1.9572
Epoch 7/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 0.3132 - loss: 1.9066 - val_accuracy: 0.2882 - val_loss: 2.0028
Epoch 8/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 0.3111 - loss: 1.9462 - val_accuracy:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

def inception_module(x, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
    # شاخه اول: کانولوشن 1x1
    conv1 = Conv2D(f1, (1, 1), padding='same', activation='relu')(x)

    # شاخه دوم: کانولوشن 1x1 و سپس 3x3
    conv3 = Conv2D(f2_in, (1, 1), padding='same', activation='relu')(x)
    conv3 = Conv2D(f2_out, (3, 3), padding='same', activation='relu')(conv3)

    # شاخه سوم: کانولوشن 1x1 و سپس 5x5
    conv5 = Conv2D(f3_in, (1, 1), padding='same', activation='relu')(x)
    conv5 = Conv2D(f3_out, (5, 5), padding='same', activation='relu')(conv5)

    # شاخه چهارم: MaxPooling و سپس کانولوشن 1x1
    pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool = Conv2D(f4_out, (1, 1), padding='same', activation='relu')(pool)

    # ترکیب خروجی هر چهار شاخه
    output = Concatenate(axis=-1)([conv1, conv3, conv5, pool])
    return output

def build_mini_googlenet(input_shape, num_classes):
    # ورودی مدل

    # data_augmentation_layer = tf.keras.Sequential([
    #     tf.keras.layers.RandomFlip("horizontal"),
    #     tf.keras.layers.RandomRotation(0.1),
    #     tf.keras.layers.RandomZoom(0.1),
    # ], name="data_augmentation")

    data_augmentation_layer = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(0.8),
        tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2), # جابجایی بیشتر
        tf.keras.layers.RandomZoom(0.2),
        tf.keras.layers.RandomBrightness(factor=0.3),
        tf.keras.layers.RandomContrast(0.2),
    ], name="data_augmentation")

    input_layer = Input(shape=input_shape)
    x = data_augmentation_layer(input_layer)

    # لایه‌های اولیه (Stem)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_layer)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = Conv2D(192, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # استفاده از دو ماژول اینسپشن پشت سر هم
    x = inception_module(x, f1=64, f2_in=96, f2_out=128, f3_in=16, f3_out=32, f4_out=32)
    x = inception_module(x, f1=128, f2_in=128, f2_out=192, f3_in=32, f3_out=96, f4_out=64)

    # لایه‌های نهایی برای طبقه‌بندی
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    # ساخت و بازگرداندن مدل نهایی
    model = Model(inputs=input_layer, outputs=output_layer, name="mini_googlenet")
    return model

# تعریف ابعاد ورودی و تعداد کلاس‌ها
input_shape = (224, 224, 3) # ابعاد استاندارد برای GoogLeNet
num_classes = 10 # برای مثال

# ساخت مدل
model = build_mini_googlenet(input_shape, num_classes)

# نمایش خلاصه معماری
model.summary()

Model: "mini_googlenet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_50 (Conv2D)  │ (None, 112, 112,  │      9,472 │ input_layer_12[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_18    │ (None, 56, 56,    │          0 │ conv2d_50[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_51 (Conv2D)  │ (None, 56, 56,    │    110,784 │ max_pooling2d_18… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_19    │ (None, 28, 28,    │          0 │ conv2d_51[0][0]   │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_53 (Conv2D)  │ (None, 28, 28,    │     18,528 │ max_pooling2d_19… │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_55 (Conv2D)  │ (None, 28, 28,    │      3,088 │ max_pooling2d_19… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_20    │ (None, 28, 28,    │          0 │ max_pooling2d_19… │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_52 (Conv2D)  │ (None, 28, 28,    │     12,352 │ max_pooling2d_19… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_54 (Conv2D)  │ (None, 28, 28,    │    110,720 │ conv2d_53[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_56 (Conv2D)  │ (None, 28, 28,    │     12,832 │ conv2d_55[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_57 (Conv2D)  │ (None, 28, 28,    │      6,176 │ max_pooling2d_20… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 28, 28,    │          0 │ conv2d_52[0][0],  │
│ (Concatenate)       │ 256)              │            │ conv2d_54[0][0],  │
│                     │                   │            │ conv2d_56[0][0],  │
│                     │                   │            │ conv2d_57[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_59 (Conv2D)  │ (None, 28, 28,    │     32,896 │ concatenate_4[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_61 (Conv2D)  │ (None, 28, 28,    │      8,224 │ concatenate_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_21    │ (None, 28, 28,    │          0 │ concatenate_4[0]… │
│ (MaxPooling2D)      │ 256)              │            │                 

 Total params: 677,498 (2.58 MB)

 Trainable params: 677,498 (2.58 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# The input shape should match your data loading step
input_shape = (256, 256, 3)
# It's also good practice to get the number of classes directly from the dataset
num_classes = 17

# Now, build the model with the CORRECT input shape
model = build_mini_googlenet(input_shape, num_classes)

# Compile the model again
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# And now, fit the model. It will work.
model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=50
)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 28s 381ms/step - accuracy: 0.0572 - loss: 10.4291 - val_accuracy: 0.1412 - val_loss: 2.7257
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 20s 69ms/step - accuracy: 0.1423 - loss: 2.6252 - val_accuracy: 0.1706 - val_loss: 2.4886
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - accuracy: 0.1932 - loss: 2.4146 - val_accuracy: 0.1941 - val_loss: 2.4661
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 92ms/step - accuracy: 0.2571 - loss: 2.2163 - val_accuracy: 0.3176 - val_loss: 2.1281
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 0.3206 - loss: 2.0384 - val_accuracy: 0.3235 - val_loss: 2.1442
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 0.3350 - loss: 1.9873 - val_accuracy: 0.3529 - val_loss: 2.0313
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - accuracy: 0.3908 - loss: 1.8948 - val_accuracy: 0.3588 - val_loss: 2.0090
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - accuracy: 0.4344 - loss: 1.7484 - val_accuracy: 0.3294

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Dense, Dropout, Activation
from tensorflow.keras.models import Model

# ---------------------------------------------------------------------------
# ۱. بارگذاری داده‌ها
# ---------------------------------------------------------------------------
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
TRAIN_DIR = '17flowerclasses/train'
TEST_DIR = '17flowerclasses/test'

# بارگذاری داده‌های آموزشی و اعتبارسنجی
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

# بارگذاری داده‌های تست
test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

# گرفتن تعداد کلاس‌ها قبل از بهینه‌سازی
NUM_CLASSES = len(train_ds.class_names)

# بهینه‌سازی عملکرد مجموعه داده‌ها
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

heavy_data_augmentation = tf.keras.Sequential(
  [
    # این لایه‌ها را به ترتیب اعمال می‌کنیم
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.4), # چرخش با زاویه بیشتر
    tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2), # جابجایی بیشتر
    tf.keras.layers.RandomZoom(height_factor=0.3, width_factor=0.3), # زوم بیشتر

    # تغییرات رنگ و نور
    tf.keras.layers.RandomBrightness(factor=0.3),
    # tf.keras.layers.RandomContrast(factor=0.3),

    # این لایه به صورت تصادفی بخشی از تصویر را سیاه می‌کند (بسیار موثر)
    # layers.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3)) # در نسخه‌های جدیدتر TF
  ],
  name="heavy_data_augmentation"
)

# ---------------------------------------------------------------------------
# ۲. ساخت معماری سفارشی قدرتمند (VGG-style)
# ---------------------------------------------------------------------------

def build_custom_cnn(input_shape, num_classes):
    # تعریف لایه‌های افزایش داده
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(0.8),
        tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2), # جابجایی بیشتر
        tf.keras.layers.RandomZoom(0.2),
        tf.keras.layers.RandomBrightness(factor=0.3),
        tf.keras.layers.RandomContrast(0.2),
    ], name="data_augmentation")

    input_layer = Input(shape=input_shape)
    x = data_augmentation(input_layer)

    # Block 1
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Block 2
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Block 3
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # لایه‌های نهایی برای طبقه‌بندی
    x = GlobalAveragePooling2D()(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer, name="custom_vgg_style_cnn")
    return model

# ---------------------------------------------------------------------------
# ۳. ساخت، کامپایل و آموزش مدل
# ---------------------------------------------------------------------------

INPUT_SHAPE = (IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
model = build_custom_cnn(INPUT_SHAPE, NUM_CLASSES)
model.summary()

# تعریف Callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# کامپایل کردن مدل
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# شروع فرآیند آموزش
print("\nStarting model training...")
history = model.fit(
    train_ds,
    epochs=100, # اجازه می‌دهیم مدل تا ۱۰۰ ایپاک آموزش ببیند (EarlyStopping آن را زودتر متوقف خواهد کرد)
    validation_data=val_ds,
    callbacks=[reduce_lr, early_stopping]
)

# ---------------------------------------------------------------------------
# ۴. ارزیابی نهایی مدل
# ---------------------------------------------------------------------------
print("\nEvaluating final model on the test set...")
results = model.evaluate(test_ds)
print(f"\nTest Loss: {results[0]:.4f}")
print(f"Test Accuracy: {results[1]:.4f}")


Found 1190 files belonging to 17 classes.
Using 952 files for training.
Found 1190 files belonging to 17 classes.
Using 238 files for validation.
Found 170 files belonging to 17 classes.


Model: "custom_vgg_style_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 56, 56, 128)    │             

 Total params: 365,617 (1.39 MB)

 Trainable params: 363,697 (1.39 MB)

 Non-trainable params: 1,920 (7.50 KB)


Starting model training...
Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 15s 297ms/step - accuracy: 0.1415 - loss: 3.0402 - val_accuracy: 0.0546 - val_loss: 5.2512 - learning_rate: 0.0010
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 271ms/step - accuracy: 0.2833 - loss: 2.3216 - val_accuracy: 0.0924 - val_loss: 3.7589 - learning_rate: 0.0010
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 264ms/step - accuracy: 0.3226 - loss: 2.1597 - val_accuracy: 0.1471 - val_loss: 2.8019 - learning_rate: 0.0010
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 266ms/step - accuracy: 0.3551 - loss: 1.9948 - val_accuracy: 0.2059 - val_loss: 3.1818 - learning_rate: 0.0010
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 265ms/step - accuracy: 0.3536 - loss: 1.9368 - val_accuracy: 0.2479 - val_loss: 2.4925 - learning_rate: 0.0010
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 265ms/step - accuracy: 0.4112 - loss: 1.7274 - val_accuracy: 0.2857 - val_loss: 2.3536 - learning_rate: 0.0010
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 263ms/s

In [ ]:
history = model.fit(
    train_ds,
    epochs=30, # اجازه می‌دهیم مدل تا ۱۰۰ ایپاک آموزش ببیند (EarlyStopping آن را زودتر متوقف خواهد کرد)
    validation_data=val_ds,
)

Epoch 1/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 258ms/step - accuracy: 0.6406 - loss: 1.0911 - val_accuracy: 0.6303 - val_loss: 1.1870
Epoch 2/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 260ms/step - accuracy: 0.6397 - loss: 1.0619 - val_accuracy: 0.6218 - val_loss: 1.1911
Epoch 3/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 262ms/step - accuracy: 0.6359 - loss: 1.0372 - val_accuracy: 0.6134 - val_loss: 1.1994
Epoch 4/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 263ms/step - accuracy: 0.6213 - loss: 1.0951 - val_accuracy: 0.6008 - val_loss: 1.2187
Epoch 5/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 264ms/step - accuracy: 0.6674 - loss: 0.9998 - val_accuracy: 0.5924 - val_loss: 1.2290
Epoch 6/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 264ms/step - accuracy: 0.6423 - loss: 1.0963 - val_accuracy: 0.5966 - val_loss: 1.2273
Epoch 7/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 265ms/step - accuracy: 0.6532 - loss: 1.0137 - val_accuracy: 0.5966 - val_loss: 1.2240
Epoch 8/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 271ms/step - accuracy: 0.6737 - loss: 1.0115 - val_accuracy: 